In [ ]:
path = "/content/drive/MyDrive/AI Specialist Projects (Support)/NLP Data Project"
import os
os.chdir(path)
%pwd

'/content/drive/MyDrive/AI Specialist Projects (Support)/NLP Data Project'

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import tensorflow.keras.utils as ku 
import pandas as pd
import numpy as np
import string, os 
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
hotel_df = pd.read_csv('Seattle_Hotels.csv', encoding="latin-1")
all_descriptions = list(hotel_df.desc.values)

In [ ]:
len(all_descriptions)

152

In [ ]:
corpus = [x for x in all_descriptions]
corpus[:1]

["Located on the southern tip of Lake Union, the Hilton Garden Inn Seattle Downtown hotel is perfectly located for business and leisure. \nThe neighborhood is home to numerous major international companies including Amazon, Google and the Bill & Melinda Gates Foundation. A wealth of eclectic restaurants and bars make this area of Seattle one of the most sought out by locals and visitors. Our proximity to Lake Union allows visitors to take in some of the Pacific Northwest's majestic scenery and enjoy outdoor activities like kayaking and sailing. over 2,000 sq. ft. of versatile space and a complimentary business center. State-of-the-art A/V technology and our helpful staff will guarantee your conference, cocktail reception or wedding is a success. Refresh in the sparkling saltwater pool, or energize with the latest equipment in the 24-hour fitness center. Tastefully decorated and flooded with natural light, our guest rooms and suites offer everything you need to relax and stay productive

In [ ]:
t = Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, document_count=0)
t.fit_on_texts(corpus)

In [ ]:
# A dictionary of words and their counts.
print(t.word_counts)

# A dictionary of words and how many documents each appeared in.
print(t.word_docs)

# An integer count of the total number of documents that were used to fit the Tokenizer (i.e. total number of documents)
print(t.document_count)

# A dictionary of words and their uniquely assigned integers.
print(t.word_index)

OrderedDict([('located', 107), ('on', 128), ('the', 1245), ('southern', 1), ('tip', 1), ('of', 531), ('lake', 41), ('union', 32), ('hilton', 11), ('garden', 11), ('inn', 83), ('seattle', 470), ('downtown', 131), ('hotel', 280), ('is', 269), ('perfectly', 6), ('for', 215), ('business', 87), ('and', 1055), ('leisure', 18), ('neighborhood', 32), ('home', 57), ('to', 471), ('numerous', 1), ('major', 12), ('international', 32), ('companies', 6), ('including', 47), ('amazon', 19), ('google', 5), ('bill', 4), ('melinda', 4), ('gates', 5), ('foundation', 4), ('a', 611), ('wealth', 1), ('eclectic', 8), ('restaurants', 35), ('bars', 7), ('make', 43), ('this', 63), ('area', 43), ('one', 74), ('most', 40), ('sought', 1), ('out', 23), ('by', 71), ('locals', 5), ('visitors', 4), ('our', 355), ('proximity', 7), ('allows', 3), ('take', 31), ('in', 443), ('some', 22), ('pacific', 42), ("northwest's", 1), ('majestic', 4), ('scenery', 2), ('enjoy', 93), ('outdoor', 23), ('activities', 8), ('like', 44), (

In [ ]:
print('Found %s unique tokens.' % len(t.word_index))

Found 3420 unique tokens.


In [ ]:
# Tokenization
t = Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, document_count=0)

def get_sequence_of_tokens(corpus):
    t.fit_on_texts(corpus)
    total_words = len(t.word_index) + 1
    
    input_sequences = []
    for line in corpus:
        token_list = t.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
            
    return input_sequences, total_words
    
input_sequences, total_words = get_sequence_of_tokens(corpus)

In [ ]:
input_sequences[:10]

[[24, 21],
 [24, 21, 1],
 [24, 21, 1, 1734],
 [24, 21, 1, 1734, 1735],
 [24, 21, 1, 1734, 1735, 4],
 [24, 21, 1, 1734, 1735, 4, 81],
 [24, 21, 1, 1734, 1735, 4, 81, 111],
 [24, 21, 1, 1734, 1735, 4, 81, 111, 1],
 [24, 21, 1, 1734, 1735, 4, 81, 111, 1, 330],
 [24, 21, 1, 1734, 1735, 4, 81, 111, 1, 330, 331]]

In [ ]:
total_words

3421

In [ ]:
# pad sequences 
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen = max_sequence_len, padding = 'pre'))
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes = total_words)
    
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(input_sequences)

In [ ]:
def create_model(max_sequence_len, total_words):
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=max_sequence_len - 1))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 505, 10)           34210     
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 3421)              345521    
                                                                 
Total params: 424,131
Trainable params: 424,131
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
756/756 [==============================] - 57s 76ms/step - loss: 6.4933
Epoch 2/100
756/756 [==============================] - 56s 75ms/step - loss: 6.3214
Epoch 3/100
756/756 [==============================] - 56s 74ms/step - loss: 6.1306
Epoch 4/100
756/756 [==============================] - 56s 74ms/step - loss: 5.9241
Epoch 5/100
756/756 [==============================] - 56s 74ms/step - loss: 5.7295
Epoch 6/100
756/756 [==============================] - 56s 74ms/step - loss: 5.5334
Epoch 7/100
756/756 [==============================] - 56s 74ms/step - loss: 5.3289
Epoch 8/100
756/756 [==============================] - 56s 74ms/step - loss: 5.1326
Epoch 9/100
756/756 [==============================] - 56s 74ms/step - loss: 4.9427
Epoch 10/100
756/756 [==============================] - 57s 75ms/step - loss: 4.7643
Epoch 11/100
756/756 [==============================] - 56s 75ms/step - loss: 4.5966
Epoch 12/100
756/756 [==============================] - 56s 75ms/step - lo

In [ ]:
# model.save("NLP_Generation_Model")

INFO:tensorflow:Assets written to: NLP_Generation_Model/assets


INFO:tensorflow:Assets written to: NLP_Generation_Model/assets


In [ ]:
import numpy as np
def generate_text(seed_text, next_words, model, max_seq_len):
    for _ in range(next_words):
        token_list = t.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        
        # predicted = model.predict_classes(token_list, verbose=0)
        predict_x = model.predict(token_list, verbose=0)
        predicted = np.argmax(predict_x,axis=1)

        output_word = ''
        
        for word,index in t.word_index.items():
            if index == predicted:
                output_word = word
                break
                
        seed_text = seed_text + " " + output_word
        
    return seed_text.title()

In [ ]:
print(generate_text("hilton seattle downtown", 100, model, max_sequence_len))
print()
print(generate_text("best western seattle airport hotel", 200, model, max_sequence_len))
print()
print(generate_text('located in the heart of downtown seattle', 300, model, max_sequence_len))

Hilton Seattle Downtown Seattle Is A Better Question By Its Shared View Of The Range Of The Floor Of The Pacific Northwest Plan A Fresh Metropolis With Snow Capped On The Experience The Experience The Fireside Room That Is A Unique Club Spa Or A Clean Style With The Contemporary Area At The Grand Salon Hotel Is The Contemporary The Library Is Just A 1 Distance To The Best For A Spacious Hotel Located At The Best Western Seattle Airport Is The Space Needle Pike Place Market And The Space Needle Pike Place Market And The Space Needle Pike Place Market And The

Best Western Seattle Airport Hotel America Block From The Seattle Waterfront And Glimmering The Space Needle We Are A Short Walk Away From The Space Needle And 5Th Avenue And Spring Hotel The Seattle Art Museum And Within Walking Distance Of Seattle And Elliott Ferry Terminal The Halcyon Inn Of Pike Place Market Seattle Art And The Space Needle If You Are A Short Drive To The Seattle Hotels Of Seattle And Ideal The Refined Location